In [14]:
import pandas as pd
import json
import requests

In [15]:
token = json
with open('API_tokens.json') as f:
    token = json.load(f)

In [16]:
app_id = token['Client Id']
app_key = token['Client Secret']
auth_url="https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"
url = "https://tdx.transportdata.tw/api/basic/v2/Bike/Station/City/Tainan"

In [17]:
class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        content_type = 'application/x-www-form-urlencoded'
        grant_type = 'client_credentials'

        return{
            'content-type' : content_type,
            'grant_type' : grant_type,
            'client_id' : self.app_id,
            'client_secret' : self.app_key
        }
class data():

    def __init__(self, app_id, app_key, auth_response):
        self.app_id = app_id
        self.app_key = app_key
        self.auth_response = auth_response

    def get_data_header(self):
        auth_JSON = json.loads(self.auth_response.text)
        access_token = auth_JSON.get('access_token')

        return{
            'authorization': 'Bearer '+access_token
        }

In [18]:
if __name__ == '__main__':
    a = Auth(app_id, app_key)
    auth_response = requests.post(auth_url, a.get_auth_header())
    d = data(app_id, app_key, auth_response)
    data_response = requests.get(url, headers=d.get_data_header())    
    # print(auth_response.text)
    print(data_response.text)

[{"StationUID":"TNN501305001","StationID":"501305001","AuthorityID":"TNN","StationName":{"Zh_tw":"YouBike2.0_民生轉運站","En":"YouBike2.0_Minsheng Transfer Station"},"StationPosition":{"PositionLon":120.18759,"PositionLat":22.99781,"GeoHash":"wsj6yftgv"},"StationAddress":{"Zh_tw":"民生路二段416號旁","En":"No. 416, Sec. 2, Minsheng Rd."},"BikesCapacity":26,"ServiceType":2,"SrcUpdateTime":"2023-05-19T14:32:15+08:00","UpdateTime":"2023-05-19T14:38:36+08:00"},{"StationUID":"TNN501305002","StationID":"501305002","AuthorityID":"TNN","StationName":{"Zh_tw":"YouBike2.0_海安民族","En":"YouBike2.0_Hai'an Minzu Rd. Intersection"},"StationPosition":{"PositionLon":120.19767,"PositionLat":22.99841,"GeoHash":"wsj6z4tr2"},"StationAddress":{"Zh_tw":"海安路二段297號前","En":"No. 297, Sec. 2, Hai'an Rd."},"BikesCapacity":19,"ServiceType":2,"SrcUpdateTime":"2023-05-19T14:10:23+08:00","UpdateTime":"2023-05-19T14:38:36+08:00"},{"StationUID":"TNN501305003","StationID":"501305003","AuthorityID":"TNN","StationName":{"Zh_tw":"YouBike

In [19]:
data_list = json.loads(data_response.text)
df = pd.read_json(json.dumps(data_list))
display(df.head())

,StationUID,StationID,AuthorityID,StationName,StationPosition,StationAddress,BikesCapacity,ServiceType,SrcUpdateTime,UpdateTime
0,TNN501305001,501305001,TNN,"{'Zh_tw': 'YouBike2.0_民生轉運站', 'En': 'YouBike2....","{'PositionLon': 120.18759, 'PositionLat': 22.9...","{'Zh_tw': '民生路二段416號旁', 'En': 'No. 416, Sec. 2...",26,2,2023-05-19T14:32:15+08:00,2023-05-19T14:38:36+08:00
1,TNN501305002,501305002,TNN,"{'Zh_tw': 'YouBike2.0_海安民族', 'En': 'YouBike2.0...","{'PositionLon': 120.19767, 'PositionLat': 22.9...","{'Zh_tw': '海安路二段297號前', 'En': 'No. 297, Sec. 2...",19,2,2023-05-19T14:10:23+08:00,2023-05-19T14:38:36+08:00
2,TNN501305003,501305003,TNN,"{'Zh_tw': 'YouBike2.0_海安正興', 'En': 'YouBike2.0...","{'PositionLon': 120.19644, 'PositionLat': 22.9...","{'Zh_tw': '海安路二段90號對側', 'En': 'No. 90, Sec. 2,...",40,2,2023-05-19T13:50:23+08:00,2023-05-19T14:38:36+08:00
3,TNN501305004,501305004,TNN,"{'Zh_tw': 'YouBike2.0_赤崁樓', 'En': 'YouBike2.0_...","{'PositionLon': 120.20227, 'PositionLat': 22.9...","{'Zh_tw': '赤嵌街9號(對側人行道)', 'En': 'No. 9, Chikan...",45,2,2023-05-19T14:37:15+08:00,2023-05-19T14:38:36+08:00
4,TNN501305005,501305005,TNN,"{'Zh_tw': 'YouBike2.0_家庭教育中心', 'En': 'YouBike2...","{'PositionLon': 120.20639, 'PositionLat': 22.9...","{'Zh_tw': '公園路/公園路175巷(西北側)', 'En': 'Ln. 175, ...",20,2,2023-05-19T14:29:15+08:00,2023-05-19T14:38:36+08:00


In [20]:
df.to_csv('bike_raw.csv', encoding='utf-8', index=False)

In [21]:
df['StationName_Zh_tw'] = df['StationName'].apply(lambda x: x['Zh_tw'].split('_')[1])
df['StationName_En'] = df['StationName'].apply(lambda x: x['En'].split('_')[1])
display(df.head())

,StationUID,StationID,AuthorityID,StationName,StationPosition,StationAddress,BikesCapacity,ServiceType,SrcUpdateTime,UpdateTime,StationName_Zh_tw,StationName_En
0,TNN501305001,501305001,TNN,"{'Zh_tw': 'YouBike2.0_民生轉運站', 'En': 'YouBike2....","{'PositionLon': 120.18759, 'PositionLat': 22.9...","{'Zh_tw': '民生路二段416號旁', 'En': 'No. 416, Sec. 2...",26,2,2023-05-19T14:32:15+08:00,2023-05-19T14:38:36+08:00,民生轉運站,Minsheng Transfer Station
1,TNN501305002,501305002,TNN,"{'Zh_tw': 'YouBike2.0_海安民族', 'En': 'YouBike2.0...","{'PositionLon': 120.19767, 'PositionLat': 22.9...","{'Zh_tw': '海安路二段297號前', 'En': 'No. 297, Sec. 2...",19,2,2023-05-19T14:10:23+08:00,2023-05-19T14:38:36+08:00,海安民族,Hai'an Minzu Rd. Intersection
2,TNN501305003,501305003,TNN,"{'Zh_tw': 'YouBike2.0_海安正興', 'En': 'YouBike2.0...","{'PositionLon': 120.19644, 'PositionLat': 22.9...","{'Zh_tw': '海安路二段90號對側', 'En': 'No. 90, Sec. 2,...",40,2,2023-05-19T13:50:23+08:00,2023-05-19T14:38:36+08:00,海安正興,Hai'an Zhengxing St. Intetersection
3,TNN501305004,501305004,TNN,"{'Zh_tw': 'YouBike2.0_赤崁樓', 'En': 'YouBike2.0_...","{'PositionLon': 120.20227, 'PositionLat': 22.9...","{'Zh_tw': '赤嵌街9號(對側人行道)', 'En': 'No. 9, Chikan...",45,2,2023-05-19T14:37:15+08:00,2023-05-19T14:38:36+08:00,赤崁樓,Chihkan Tower
4,TNN501305005,501305005,TNN,"{'Zh_tw': 'YouBike2.0_家庭教育中心', 'En': 'YouBike2...","{'PositionLon': 120.20639, 'PositionLat': 22.9...","{'Zh_tw': '公園路/公園路175巷(西北側)', 'En': 'Ln. 175, ...",20,2,2023-05-19T14:29:15+08:00,2023-05-19T14:38:36+08:00,家庭教育中心,Family Education Center


In [22]:
df['StationAddress_Zh_tw'] = df['StationAddress'].apply(lambda x: x['Zh_tw'])
df['StationAddress_En'] = df['StationAddress'].apply(lambda x: x['En'])
display(df.head())

,StationUID,StationID,AuthorityID,StationName,StationPosition,StationAddress,BikesCapacity,ServiceType,SrcUpdateTime,UpdateTime,StationName_Zh_tw,StationName_En,StationAddress_Zh_tw,StationAddress_En
0,TNN501305001,501305001,TNN,"{'Zh_tw': 'YouBike2.0_民生轉運站', 'En': 'YouBike2....","{'PositionLon': 120.18759, 'PositionLat': 22.9...","{'Zh_tw': '民生路二段416號旁', 'En': 'No. 416, Sec. 2...",26,2,2023-05-19T14:32:15+08:00,2023-05-19T14:38:36+08:00,民生轉運站,Minsheng Transfer Station,民生路二段416號旁,"No. 416, Sec. 2, Minsheng Rd."
1,TNN501305002,501305002,TNN,"{'Zh_tw': 'YouBike2.0_海安民族', 'En': 'YouBike2.0...","{'PositionLon': 120.19767, 'PositionLat': 22.9...","{'Zh_tw': '海安路二段297號前', 'En': 'No. 297, Sec. 2...",19,2,2023-05-19T14:10:23+08:00,2023-05-19T14:38:36+08:00,海安民族,Hai'an Minzu Rd. Intersection,海安路二段297號前,"No. 297, Sec. 2, Hai'an Rd."
2,TNN501305003,501305003,TNN,"{'Zh_tw': 'YouBike2.0_海安正興', 'En': 'YouBike2.0...","{'PositionLon': 120.19644, 'PositionLat': 22.9...","{'Zh_tw': '海安路二段90號對側', 'En': 'No. 90, Sec. 2,...",40,2,2023-05-19T13:50:23+08:00,2023-05-19T14:38:36+08:00,海安正興,Hai'an Zhengxing St. Intetersection,海安路二段90號對側,"No. 90, Sec. 2, Hai'an Rd.(Opposite)"
3,TNN501305004,501305004,TNN,"{'Zh_tw': 'YouBike2.0_赤崁樓', 'En': 'YouBike2.0_...","{'PositionLon': 120.20227, 'PositionLat': 22.9...","{'Zh_tw': '赤嵌街9號(對側人行道)', 'En': 'No. 9, Chikan...",45,2,2023-05-19T14:37:15+08:00,2023-05-19T14:38:36+08:00,赤崁樓,Chihkan Tower,赤嵌街9號(對側人行道),"No. 9, Chikan St.(Opposite)"
4,TNN501305005,501305005,TNN,"{'Zh_tw': 'YouBike2.0_家庭教育中心', 'En': 'YouBike2...","{'PositionLon': 120.20639, 'PositionLat': 22.9...","{'Zh_tw': '公園路/公園路175巷(西北側)', 'En': 'Ln. 175, ...",20,2,2023-05-19T14:29:15+08:00,2023-05-19T14:38:36+08:00,家庭教育中心,Family Education Center,公園路/公園路175巷(西北側),"Ln. 175, Gongyuan Rd. & Gongyuan Rd. Intersection"


In [23]:
df['PositionLon'] = df['StationPosition'].apply(lambda x: x['PositionLon'])
df['PositionLat'] = df['StationPosition'].apply(lambda x: x['PositionLat'])
df['GeoHash'] = df['StationPosition'].apply(lambda x: x['GeoHash'])
display(df.head())

,StationUID,StationID,AuthorityID,StationName,StationPosition,StationAddress,BikesCapacity,ServiceType,SrcUpdateTime,UpdateTime,StationName_Zh_tw,StationName_En,StationAddress_Zh_tw,StationAddress_En,PositionLon,PositionLat,GeoHash
0,TNN501305001,501305001,TNN,"{'Zh_tw': 'YouBike2.0_民生轉運站', 'En': 'YouBike2....","{'PositionLon': 120.18759, 'PositionLat': 22.9...","{'Zh_tw': '民生路二段416號旁', 'En': 'No. 416, Sec. 2...",26,2,2023-05-19T14:32:15+08:00,2023-05-19T14:38:36+08:00,民生轉運站,Minsheng Transfer Station,民生路二段416號旁,"No. 416, Sec. 2, Minsheng Rd.",120.18759,22.99781,wsj6yftgv
1,TNN501305002,501305002,TNN,"{'Zh_tw': 'YouBike2.0_海安民族', 'En': 'YouBike2.0...","{'PositionLon': 120.19767, 'PositionLat': 22.9...","{'Zh_tw': '海安路二段297號前', 'En': 'No. 297, Sec. 2...",19,2,2023-05-19T14:10:23+08:00,2023-05-19T14:38:36+08:00,海安民族,Hai'an Minzu Rd. Intersection,海安路二段297號前,"No. 297, Sec. 2, Hai'an Rd.",120.19767,22.99841,wsj6z4tr2
2,TNN501305003,501305003,TNN,"{'Zh_tw': 'YouBike2.0_海安正興', 'En': 'YouBike2.0...","{'PositionLon': 120.19644, 'PositionLat': 22.9...","{'Zh_tw': '海安路二段90號對側', 'En': 'No. 90, Sec. 2,...",40,2,2023-05-19T13:50:23+08:00,2023-05-19T14:38:36+08:00,海安正興,Hai'an Zhengxing St. Intetersection,海安路二段90號對側,"No. 90, Sec. 2, Hai'an Rd.(Opposite)",120.19644,22.99461,wsj6z4h3k
3,TNN501305004,501305004,TNN,"{'Zh_tw': 'YouBike2.0_赤崁樓', 'En': 'YouBike2.0_...","{'PositionLon': 120.20227, 'PositionLat': 22.9...","{'Zh_tw': '赤嵌街9號(對側人行道)', 'En': 'No. 9, Chikan...",45,2,2023-05-19T14:37:15+08:00,2023-05-19T14:38:36+08:00,赤崁樓,Chihkan Tower,赤嵌街9號(對側人行道),"No. 9, Chikan St.(Opposite)",120.20227,22.99752,wsj6z68d7
4,TNN501305005,501305005,TNN,"{'Zh_tw': 'YouBike2.0_家庭教育中心', 'En': 'YouBike2...","{'PositionLon': 120.20639, 'PositionLat': 22.9...","{'Zh_tw': '公園路/公園路175巷(西北側)', 'En': 'Ln. 175, ...",20,2,2023-05-19T14:29:15+08:00,2023-05-19T14:38:36+08:00,家庭教育中心,Family Education Center,公園路/公園路175巷(西北側),"Ln. 175, Gongyuan Rd. & Gongyuan Rd. Intersection",120.20639,22.99715,wsj6z6e85


In [24]:
columns_to_drop = ['StationUID', 'AuthorityID', 'StationName', 'StationPosition', 'StationAddress',
                   'ServiceType', 'SrcUpdateTime', 'UpdateTime', 'StationName_En', 'StationAddress_En']
df.drop(labels=columns_to_drop, axis=1, inplace=True)
df = df.rename(columns={
    'StationName_Zh_tw': 'StationName',
    'StationAddress_Zh_tw': 'StationAddress'
})
display(df.head())

,StationID,BikesCapacity,StationName,StationAddress,PositionLon,PositionLat,GeoHash
0,501305001,26,民生轉運站,民生路二段416號旁,120.18759,22.99781,wsj6yftgv
1,501305002,19,海安民族,海安路二段297號前,120.19767,22.99841,wsj6z4tr2
2,501305003,40,海安正興,海安路二段90號對側,120.19644,22.99461,wsj6z4h3k
3,501305004,45,赤崁樓,赤嵌街9號(對側人行道),120.20227,22.99752,wsj6z68d7
4,501305005,20,家庭教育中心,公園路/公園路175巷(西北側),120.20639,22.99715,wsj6z6e85


In [25]:
df.to_csv('bike.csv', encoding='utf-8', index=False)